In [6]:
!pip3 install pyspark

  Using cached pyspark-3.4.1.tar.gz (310.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=4cb9d712197667e7447d716c60bcea59aed6b9de0be5e1053d69fb1f59a52e34
  Stored in directory: /Users/yinka/Library/Caches/pip/wheels/2b/9a/39/d8019ffbfb76a39433455e3d5799e94d3e3cae8f41229f6bf8
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('NewManager').getOrCreate()

23/09/09 08:21:45 WARN Utils: Your hostname, Yinkas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.51 instead (on interface en0)
23/09/09 08:21:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/09 08:21:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/09 08:21:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# understand the dataset

In [3]:
#reading from the text file
log_df = spark.read.csv('log.txt', sep='|', inferSchema=True)

In [4]:
#examine the dataset
log_df.show(30)

+--------------------+-------+-----------+--------------------+-------------+--------------------+
|                 _c0|    _c1|        _c2|                 _c3|          _c4|                 _c5|
+--------------------+-------+-----------+--------------------+-------------+--------------------+
|Data quality assu...|success|89328463487|         Florida;USA|1599093305000|464f:bdbd:1d:535a...|
|Designing and imp...|success|89328463488|      Ontario;Canada|1589218599000|1366:c314:8d40:42...|
|Extracting data f...|success|89328463489|      California;USA|1595463446000|1ffa:cdd1:7a2b:a7...|
|Loading data into...|success|89328463490|         Florida;USA|1605932487000|8ac5:9010:f24c:2f...|
|Data quality assu...|success|89328463491|Queensland;Australia|1587354266000|5a7d:18ec:4650:54...|
|Building data pip...|success|89328463492|        Paris;France|1600093768000|1480:2795:2b3a:42...|
|Optimizing databa...|      -|89328463493|         Tokyo;Japan|1596900215000|7f2b:df86:5279:68...|
|Data qual

In [5]:
columns = ['task', 'status', 'trackID', 'server_region', 'timestamp', 'server']

In [6]:
log_df = log_df.withColumnsRenamed({'_c0':'task', '_c1': 'status', '_c2': 'trackID', '_c3': 'server_region', '_c4': 'timestamp', '_c5':'server'})

In [7]:
log_df.show()

+--------------------+-------+-----------+--------------------+-------------+--------------------+
|                task| status|    trackID|       server_region|    timestamp|              server|
+--------------------+-------+-----------+--------------------+-------------+--------------------+
|Data quality assu...|success|89328463487|         Florida;USA|1599093305000|464f:bdbd:1d:535a...|
|Designing and imp...|success|89328463488|      Ontario;Canada|1589218599000|1366:c314:8d40:42...|
|Extracting data f...|success|89328463489|      California;USA|1595463446000|1ffa:cdd1:7a2b:a7...|
|Loading data into...|success|89328463490|         Florida;USA|1605932487000|8ac5:9010:f24c:2f...|
|Data quality assu...|success|89328463491|Queensland;Australia|1587354266000|5a7d:18ec:4650:54...|
|Building data pip...|success|89328463492|        Paris;France|1600093768000|1480:2795:2b3a:42...|
|Optimizing databa...|      -|89328463493|         Tokyo;Japan|1596900215000|7f2b:df86:5279:68...|
|Data qual

In [8]:
# look at the datatypes
log_df.dtypes

[('task', 'string'),
 ('status', 'string'),
 ('trackID', 'bigint'),
 ('server_region', 'string'),
 ('timestamp', 'bigint'),
 ('server', 'string')]

In [9]:
# obtain the shape of the file
print("shape of the record is ", log_df.count(), len(log_df.columns))

shape of the record is  1000 6


In [10]:
log_df.printSchema()

root
 |-- task: string (nullable = true)
 |-- status: string (nullable = true)
 |-- trackID: long (nullable = true)
 |-- server_region: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- server: string (nullable = true)



In [11]:
from pyspark.sql import functions as fn

In [12]:
log_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----+------+-------+-------------+---------+------+
|task|status|trackID|server_region|timestamp|server|
+----+------+-------+-------------+---------+------+
|   0|     0|      0|            0|        0|     0|
+----+------+-------+-------------+---------+------+



# Cleaning and transformation

In [13]:
# the columns have been renamed
#split the server_region column


In [13]:
#split the server_region column
# define split
split_cols = fn.split(log_df['server_region'], ';')

In [14]:
#applying split() using select()
new_log_df = log_df.select('task', 'status', 'trackID', 'server_region', 'timestamp', 'server',
                           split_cols.getItem(0).alias('state'),
                           split_cols.getItem(1).alias('country'))

In [15]:
new_log_df.show()

+--------------------+-------+-----------+--------------------+-------------+--------------------+----------+---------+
|                task| status|    trackID|       server_region|    timestamp|              server|     state|  country|
+--------------------+-------+-----------+--------------------+-------------+--------------------+----------+---------+
|Data quality assu...|success|89328463487|         Florida;USA|1599093305000|464f:bdbd:1d:535a...|   Florida|      USA|
|Designing and imp...|success|89328463488|      Ontario;Canada|1589218599000|1366:c314:8d40:42...|   Ontario|   Canada|
|Extracting data f...|success|89328463489|      California;USA|1595463446000|1ffa:cdd1:7a2b:a7...|California|      USA|
|Loading data into...|success|89328463490|         Florida;USA|1605932487000|8ac5:9010:f24c:2f...|   Florida|      USA|
|Data quality assu...|success|89328463491|Queensland;Australia|1587354266000|5a7d:18ec:4650:54...|Queensland|Australia|
|Building data pip...|success|8932846349

In [16]:
from pyspark.sql import types as dtypes

In [17]:
# convert the timestamp
new_log_df = new_log_df\
    .withColumn('time_stamp', fn.from_unixtime(fn.col('timestamp')/1000, 'yyyy-MM-dd HH:mm:ss'))\
        .withColumn('time_stamp', fn.col('time_stamp').cast(dtypes.TimestampType()))

In [18]:
new_log_df.show(truncate=False)

+---------------------------------------+-------+-----------+--------------------+-------------+---------------------------------------+----------+---------+-------------------+
|task                                   |status |trackID    |server_region       |timestamp    |server                                 |state     |country  |time_stamp         |
+---------------------------------------+-------+-----------+--------------------+-------------+---------------------------------------+----------+---------+-------------------+
|Data quality assurance                 |success|89328463487|Florida;USA         |1599093305000|464f:bdbd:1d:535a:2dad:5117:a92f:b359  |Florida   |USA      |2020-09-02 18:35:05|
|Designing and implementing data models |success|89328463488|Ontario;Canada      |1589218599000|1366:c314:8d40:4219:4a51:138f:7ce3:d244|Ontario   |Canada   |2020-05-11 11:36:39|
|Extracting data from various sources   |success|89328463489|California;USA      |1595463446000|1ffa:cdd1:7a2b

In [19]:
new_log_df.dtypes

[('task', 'string'),
 ('status', 'string'),
 ('trackID', 'bigint'),
 ('server_region', 'string'),
 ('timestamp', 'bigint'),
 ('server', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('time_stamp', 'timestamp')]

# carrying out analysis for the Data Engineer

In [20]:
#retrieving the number of countries
new_log_df2 = new_log_df.groupBy('country').count()
new_log_df2.show()

+---------+-----+
|  country|count|
+---------+-----+
|   France|  124|
|      USA|  491|
|   Canada|  124|
|    Japan|  133|
|Australia|  128|
+---------+-----+



In [21]:
# states in each country where the tasks are carried out
new_log_df3 = new_log_df.groupBy('country', 'state').count().orderBy(fn.col('count').desc())

In [22]:
new_log_df3.show()

+---------+----------+-----+
|  country|     state|count|
+---------+----------+-----+
|      USA|  New York|  145|
|      USA|   Florida|  136|
|    Japan|     Tokyo|  133|
|Australia|Queensland|  128|
|   Canada|   Ontario|  124|
|   France|     Paris|  124|
|      USA|     Texas|  114|
|      USA|California|   96|
+---------+----------+-----+



In [23]:
# task frequently carried out in each country
new_log_df4 = new_log_df.groupBy('task', 'country').count().orderBy(['country', 'count'], ascending=False)

In [24]:
new_log_df4.show(50, truncate=False)

+---------------------------------------+---------+-----+
|task                                   |country  |count|
+---------------------------------------+---------+-----+
|Extracting data from various sources   |USA      |98   |
|Loading data into a data warehouse     |USA      |58   |
|Data quality assurance                 |USA      |57   |
|Transforming data into a unified format|USA      |52   |
|Data governance                        |USA      |49   |
|Optimizing database performance        |USA      |49   |
|Building data pipelines                |USA      |47   |
|Designing and implementing data models |USA      |42   |
|Data integration                       |USA      |39   |
|Extracting data from various sources   |Japan    |25   |
|Data governance                        |Japan    |18   |
|Data quality assurance                 |Japan    |16   |
|Transforming data into a unified format|Japan    |15   |
|Building data pipelines                |Japan    |14   |
|Data integrat

In [25]:
#performance of each task
new_log_df5 = new_log_df.groupBy('task', 'status').count().orderBy(['status', 'count'], ascending=False)

In [26]:
new_log_df5.show(24, truncate=False)

+---------------------------------------+-------+-----+
|task                                   |status |count|
+---------------------------------------+-------+-----+
|Extracting data from various sources   |success|150  |
|Data governance                        |success|90   |
|Loading data into a data warehouse     |success|90   |
|Data quality assurance                 |success|87   |
|Building data pipelines                |success|76   |
|Transforming data into a unified format|success|76   |
|Data integration                       |success|74   |
|Optimizing database performance        |success|66   |
|Designing and implementing data models |success|57   |
|Extracting data from various sources   |failed |31   |
|Building data pipelines                |failed |13   |
|Data quality assurance                 |failed |13   |
|Transforming data into a unified format|failed |13   |
|Data governance                        |failed |13   |
|Designing and implementing data models |failed 

In [27]:
new_log_df.show()

+--------------------+-------+-----------+--------------------+-------------+--------------------+----------+---------+-------------------+
|                task| status|    trackID|       server_region|    timestamp|              server|     state|  country|         time_stamp|
+--------------------+-------+-----------+--------------------+-------------+--------------------+----------+---------+-------------------+
|Data quality assu...|success|89328463487|         Florida;USA|1599093305000|464f:bdbd:1d:535a...|   Florida|      USA|2020-09-02 18:35:05|
|Designing and imp...|success|89328463488|      Ontario;Canada|1589218599000|1366:c314:8d40:42...|   Ontario|   Canada|2020-05-11 11:36:39|
|Extracting data f...|success|89328463489|      California;USA|1595463446000|1ffa:cdd1:7a2b:a7...|California|      USA|2020-07-22 18:17:26|
|Loading data into...|success|89328463490|         Florida;USA|1605932487000|8ac5:9010:f24c:2f...|   Florida|      USA|2020-11-20 22:21:27|
|Data quality assu..

In [28]:
reordered_columns = ['task', 'status', 'trackID', 'server', 'state', 'country', 'time_stamp']
changed_log_df = new_log_df.select(reordered_columns)

In [30]:
changed_log_df.show(truncate=False)

+---------------------------------------+-------+-----------+---------------------------------------+----------+---------+-------------------+
|task                                   |status |trackID    |server                                 |state     |country  |time_stamp         |
+---------------------------------------+-------+-----------+---------------------------------------+----------+---------+-------------------+
|Data quality assurance                 |success|89328463487|464f:bdbd:1d:535a:2dad:5117:a92f:b359  |Florida   |USA      |2020-09-02 18:35:05|
|Designing and implementing data models |success|89328463488|1366:c314:8d40:4219:4a51:138f:7ce3:d244|Ontario   |Canada   |2020-05-11 11:36:39|
|Extracting data from various sources   |success|89328463489|1ffa:cdd1:7a2b:a7d8:ab00:eada:94b3:7a7c|California|USA      |2020-07-22 18:17:26|
|Loading data into a data warehouse     |success|89328463490|8ac5:9010:f24c:2f06:4679:aaec:a737:f003|Florida   |USA      |2020-11-20 22:21:27|